<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      tdplyr R Basics
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata Package for R (tdplyr) is an open-source R library package that combines the benefits of open-source R language environment with the massive parallel processing capabilities of Teradata Vantage, which includes the Machine Learning Engine analytic functions and the Analytics Database in-database analytic functions. Teradata Package for R allows users to develop and run R programs that take advantage of the Big Data and Machine Learning analytics capabilities of Vantage, and can be used in conjunction with open source R capabilities. Moreover, the tdplyr package conforms and works with the functions of the dbplyr package and most of the verbs of the dplyr package.</p>
  
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This notebook will cover the very basics of the tdplyr package and is a technical demonstration of different functionalities of Teradataml. This is not a business outcome type demo.  Please see the Getting Started Guide online <a href = 'https://docs.teradata.com/search/all?query=Introduction+to+Teradata+Package+for+R&filters=prodname~%2522Teradata+Package+for+R%2522&content-lang=en-US'>here.</a></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Install and load the necessary Packages and Libraries</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Installing R only needs to be done once for this environment.
<br><br>
To install the package, copy this line into the clipboard:
<blockquote><i>
Rscript -e "install.packages('tdplyr',repos=c('https://r-repo.teradata.com','https://cloud.r-project.org'))"
</i></blockquote> </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Open a terminal window by selecting File --> New --> and the select Terminal.  Then paste the line at the Command Prompt and press Enter.  This will take a few minutes.  It's finished when you are returned to the <b>(base) jovyan@de76f2e68a54:~/JupyterLabRoot$ </b> Command Prompt.
</p>

    

In [1]:
suppressWarnings({
library(tdplyr)
library(dbplyr)
library(dplyr)
library(DBI)    
    })

Registered S3 methods overwritten by 'tdplyr':
  method                     from  
  sql_query_select.Teradata  dbplyr
  sql_table_analyze.Teradata dbplyr
  sql_translation.Teradata   dbplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Help function will list all the documentation related to tdplyr package.</p>

In [2]:
help(package='tdplyr')

Documentation for package ‘tdplyr’


		Information on package ‘tdplyr’

Description:

Package:              tdplyr
Type:                 Package
Title:                Teradata R Package
Version:              17.20.00.00
Date:                 2018-12-07
Author:               Waman Joshi, Mark Sandan, Arun Kalyanasundaram,
                      Karthik Thelukuntla, Pankaj Vinod Purandare,
                      Trupti A Purohit, Amanda Moran, Jeris Alan
                      Jawahar, Adithya Avvaru, Mounika Kotha, Rohit
                      Khurd, Pavansai Kumar Alladi, Rohit Agrawal,
                      Gouri Patwardhan, Subhash Karemore, Aanchal
                      Kavedia
Maintainer:           Analytic-Clients, Support <SA230577@teradata.com>
LazyData:             yes
Depends:              R (>= 3.6.3)
Imports:              DBI (>= 1.0.0), dplyr (>= 1.1.0), dbplyr (>=
                      2.3.2), jsonlite (>= 1.7), magrittr (>= 1.5),
                      teradatasql (>= 17.20.0.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Below code is used to check the tdplyr package version installed.</p>

In [3]:
packageVersion("tdplyr")

[1] ‘17.20.0.0’

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [4]:
con <- td_create_context(host = 'host.docker.internal', uid = "demo_user", pwd = getPass("Enter your password: "), dType = "NATIVE", logmech = "TD2")

Enter your password:  ········


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Set the queryband, below code will set the queryband for the session.</p>

In [5]:
dbExecute(con,"SET query_band='DEMO=tdplyr_R_Basics.ipynb;' UPDATE FOR SESSION;") 

[1] 1

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>

In [6]:
dbExecute(con,"call get_data('DEMO_DataScienceExploration_local');") 
#takes 2min to load the data

[1] 0

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Creating a tibble from a Table in Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The tbl() function of the "dplyr" package creates a tdplyr tibble from an existing table in a database in Vantage.<br>
A tdplyr tibble is an R table that resembles an R tibble, that is, a form of data frame. The difference between a tdplyr tibble and an R tibble is that the output object of tbl() function shows that a tdplyr tibble is a remote source in a database in Vantage.</p>

In [7]:
tdf <- tbl(con, in_schema("DEMO_DataScienceExploration", "House_Prices"))
tdf

# Source:   table<"DEMO_DataScienceExploration"."House_Prices"> [?? x 21]
# Database: Teradata
        id date         price bedrooms bathrooms sqft_living sqft_lot floors
   <int64> <date>       <dbl>    <int>     <dbl>       <dbl>    <dbl>  <dbl>
 1   78876 2014-05-29 1355000        5      3.5         5960    13703    2  
 2   36889 2014-05-05  418000        3      2           1410     6030    1.5
 3   11603 2014-08-13  275000        2      1.75        2090    11317    1  
 4   79345 2014-06-25  957000        5      3.25        3160     5000    2  
 5    9054 2015-03-26  292500        4      2.5         1610     4568    2  
 6   54059 2015-02-19  700000        4      2           2230     4635    1.5
 7   17639 2014-05-12  300000        2      1.75        1360     8100    1  
 8   34340 2014-11-19 1010000        4      3.5         3350     3752    2  
 9   62175 2014-11-18  422500        4      2.5         2550     8824    2  
10   39438 2014-09-26  350000        4      1           10

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>glimpse function will give the description of columns in dataframe/tibble.</p>

In [8]:
glimpse(tdf)

Rows: ??
Columns: 21
Database: Teradata
$ id            <int64> 78876, 36889, 11603, 79345, 9054, 54059, 17639, 37358,…
$ date          <date> 2014-05-29, 2014-05-05, 2014-08-13, 2014-06-25, 2015-03…
$ price         <dbl> 1355000, 418000, 275000, 957000, 292500, 700000, 300000,…
$ bedrooms      <int> 5, 3, 2, 5, 4, 4, 2, 3, 4, 3, 4, 4, 4, 4, 3, 3, 3, 3, 2,…
$ bathrooms     <dbl> 3.50, 2.00, 1.75, 3.25, 2.50, 2.00, 1.75, 1.00, 2.50, 1.…
$ sqft_living   <dbl> 5960, 1410, 2090, 3160, 1610, 2230, 1360, 1410, 2690, 18…
$ sqft_lot      <dbl> 13703, 6030, 11317, 5000, 4568, 4635, 8100, 47916, 32780…
$ floors        <dbl> 2.0, 1.5, 1.0, 2.0, 2.0, 1.5, 1.0, 1.0, 2.0, 1.0, 2.0, 2…
$ waterfront    <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ view          <int> 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ condition     <int> 3, 4, 5, 3, 3, 5, 3, 3, 3, 3, 3, 3, 4, 4, 3, 5, 3, 4, 3,…
$ grade         <int> 10, 7, 7, 10, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 11, 8, 7,…


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Below code can be used to get the total row count of the tibble</p>

In [9]:
td_nrow(tdf)

integer64
[1] 80000

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>head will extract first few rows of the tibble.</p>

In [10]:
tdf %>% head()

# Source:   SQL [6 x 21]
# Database: Teradata
       id date        price bedrooms bathrooms sqft_living sqft_lot floors
  <int64> <date>      <dbl>    <int>     <dbl>       <dbl>    <dbl>  <dbl>
1   48023 2014-05-14 530000        4      2.75        2450    15002      1
2   25755 2015-05-07 567500        4      1.75        2190     5060      1
3     469 2015-03-31 312000        3      2.25        1470     7857      1
4   51041 2014-08-07 210000        2      0.75         840    49658      1
5   28304 2015-01-27 679950        3      2.5         2230     3939      2
6   53590 2014-06-02 559900        3      2.75        2930     5569      1
# ℹ 13 more variables: waterfront <int>, view <int>, condition <int>,
#   grade <int>, sqft_above <dbl>, sqft_basement <dbl>, yr_built <int>,
#   yr_renovated <int>, zipcode <chr>, lat <dbl>, long <dbl>,
#   sqft_living15 <dbl>, sqft_lot15 <dbl>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Aggregations</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Various aggregations are available for grouping, windowing, time series, etc.</p>

In [11]:
tdf %>% group_by(bedrooms) %>% summarize(mean = mean(price), n = n())

Warning message:
“Missing values are always removed in SQL aggregation functions.
Use `na.rm = TRUE` to silence this warning
This warning is displayed once every 8 hours.”


# Source:   SQL [?? x 3]
# Database: Teradata
   bedrooms     mean       n
      <int>    <dbl> <int64>
 1       33  640000        2
 2        4  637209.   25582
 3        7  970473.     145
 4        0  359465.      55
 5        8 1018562.      48
 6        3  466396.   36222
 7        1  317503.     736
 8        2  399480.   10227
 9        6  803804.    1056
10       10  802400       10
# ℹ more rows

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
<i>mutate(price_per_bed = price / bedrooms) </i><br>
is used to create a new column price_per_bed in the tdf table, which contains the result of dividing the price column by the bedrooms column.</p>

In [12]:
tdf <- tdf %>% mutate(price_per_bed = price / bedrooms)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Print all columns and a specified number of rows (e.g., 10)

In [13]:
print(tdf, n = 10, width = Inf)

# Source:   SQL [?? x 22]
# Database: Teradata
        id date         price bedrooms bathrooms sqft_living sqft_lot floors
   <int64> <date>       <dbl>    <int>     <dbl>       <dbl>    <dbl>  <dbl>
 1   78876 2014-05-29 1355000        5      3.5         5960    13703    2  
 2   36889 2014-05-05  418000        3      2           1410     6030    1.5
 3   11603 2014-08-13  275000        2      1.75        2090    11317    1  
 4   79345 2014-06-25  957000        5      3.25        3160     5000    2  
 5    9054 2015-03-26  292500        4      2.5         1610     4568    2  
 6   54059 2015-02-19  700000        4      2           2230     4635    1.5
 7   17639 2014-05-12  300000        2      1.75        1360     8100    1  
 8   34340 2014-11-19 1010000        4      3.5         3350     3752    2  
 9   62175 2014-11-18  422500        4      2.5         2550     8824    2  
10   39438 2014-09-26  350000        4      1           1010     6000    1  
   waterfront  view condition

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Cleanup</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [14]:
dbExecute(con,"call remove_data('DEMO_DataScienceExploration');") 

[1] 0

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>It is a good practice to remove the context that we created to connect to Vantage. The remove_context function removes the current context associated with the Vantage connection.remove_context() not only closes the connection but also garbage collects the intermediate views and tables created by tdplyr. Teradata recommends calling remove_context() to end a session, so that intermediate views and tables created by tdplyr are garbage collected.</p>

In [15]:
td_remove_context()

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Downloading and Installing the Teradata R package</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Teradata Package for R, tdplyr, is available from Teradata GitHub site <a href = 'https://github.com/Teradata/tdplyr'>here.</a> <br>
The following command should be run from a Terminal (for Linux and Mac) or Command Prompt (for Windows).  This will download and install the tdplyr package along with the dependent packages, from Amazon AWS and CRAN repository.  
<blockquote><i>
Rscript -e "install.packages('tdplyr',repos=c('https://r-repo.teradata.com','https://cloud.r-project.org'))"
</i></blockquote> </p>

    

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>